# D. State stabilization in a noisy environment with Jaynes-Cummings controls

Here the result is as follows: The algorithm optimizes the params, such that the POVM always outputs 1, implying that the measurement leaves the target state invariant. this is what we are indeed seeing when printing the measurement outcome and its probability, when batching however, the optimizer struggles to converge.

Also, lookup here is much better than nn for the same hyperparameters.

This is actually a perfect example, of feedback grape modifying the params so that a certain measurement sequence will always be output because this measurement sequence is the one that is going to lead to the best fidelity

In [41]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [42]:
from feedback_grape.fgrape import optimize_pulse
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

In [43]:
N_cav = 30

## Here, dividing alpha into real and imaginary parts complicates the optimization and converges at 0.89 while if we do not use the imaginary part it converges at 0.999

In [44]:
def qubit_unitary(alpha_re):
    alpha = alpha_re
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [45]:
def qubit_cavity_unitary(beta_re):
    beta = beta_re
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

### povm_measure_operator (callable): <br>
    - It should take a measurement outcome and list of params as input
    - The measurement outcome options are either 1 or -1

In [46]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma, delta = params
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

In [47]:
from feedback_grape.utils.states import coherent

alpha = 3
psi_target = tensor(
    coherent(N_cav, alpha)
    + coherent(N_cav, -alpha)
    + coherent(N_cav, 1j * alpha)
    + coherent(N_cav, -1j * alpha),
    basis(2),
)  # 4-legged state

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)
rho_target = psi_target @ psi_target.conj().T

In [48]:
rho_target.shape

(60, 60)

### It is important to test what the POVM probability is, to check if your state is normalized. if the probability is bounded between 0 and 1 then normalized

In [49]:
# Answer: should one normalize within the optimization just in case?
# --> no so that user is not misled into thinking that his unnormalized state
# is working properly
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho_target,
    1,
    povm_measure_operator,
    [0.058, jnp.pi / 2],
    evo_type="density",
)

Array(0.70390199, dtype=float64)

In [50]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho_target, C_target=rho_target, evo_type="density"))

1.0000000233601563


### Without dissipation

In [51]:
# Here the loss directly corressponds to the -fidelity (when converging) because log(1) is 0 and
# the algorithm is choosing params that makes the POVM generate prob = 1
from feedback_grape.fgrape import Gate, Decay

measure = Gate(
    gate=povm_measure_operator,
    initial_params=[0.058, jnp.pi / 2],  # gamma and
    measurement_flag=True,
)

qub_unitary = Gate(
    gate=qubit_unitary,
    initial_params=[jnp.pi / 3],  # alpha
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)

qub_cav = Gate(
    gate=qubit_cavity_unitary,
    initial_params=[jnp.pi / 3],  # beta
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)

for reward_weights in [[1.0,1.0],[0.0,1.0]]:
    system_params = [measure, qub_unitary, qub_cav]
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-16,
        learning_rate=0.005,
        evo_type="density",
        batch_size=1,
        eval_batch_size=16,
        eval_time_steps=5,
    )

    print(f"reward weights: {reward_weights}\n fidelity@t=2: {result.fidelity_each_timestep[2]}\n fidelity_each_timestep: {jnp.mean(jnp.array(result.fidelity_each_timestep), axis=1)}\n")

reward weights: [1.0, 1.0]
 fidelity@t=2: [0.79470343 0.79470343 0.79470343 0.79470343 0.79470343 0.79470343
 0.79470343 0.79470343 0.79470343 0.79470343 0.79470343 0.79470343
 0.79470343 0.79470343 0.79470343 0.79470343]
 fidelity_each_timestep: [1.00000002 0.99900573 0.79470343 0.3078017  0.11266678 0.045482  ]

reward weights: [0.0, 1.0]
 fidelity@t=2: [0.81325896 0.81325896 0.81325896 0.81325896 0.81325896 0.81325896
 0.81325896 0.81325896 0.81325896 0.81325896 0.81325896 0.81325896
 0.81325896 0.81325896 0.81325896 0.81325896]
 fidelity_each_timestep: [1.00000002 0.23789578 0.81325896 0.31706842 0.04127971 0.14997662]



In [52]:
result.fidelity_each_timestep

[Array([1.00000002, 1.00000002, 1.00000002, 1.00000002, 1.00000002,
        1.00000002, 1.00000002, 1.00000002, 1.00000002, 1.00000002,
        1.00000002, 1.00000002, 1.00000002, 1.00000002, 1.00000002,
        1.00000002], dtype=float64),
 Array([0.23789578, 0.23789578, 0.23789578, 0.23789578, 0.23789578,
        0.23789578, 0.23789578, 0.23789578, 0.23789578, 0.23789578,
        0.23789578, 0.23789578, 0.23789578, 0.23789578, 0.23789578,
        0.23789578], dtype=float64),
 Array([0.81325896, 0.81325896, 0.81325896, 0.81325896, 0.81325896,
        0.81325896, 0.81325896, 0.81325896, 0.81325896, 0.81325896,
        0.81325896, 0.81325896, 0.81325896, 0.81325896, 0.81325896,
        0.81325896], dtype=float64),
 Array([0.38303474, 0.11916945, 0.11916945, 0.38303474, 0.38303474,
        0.38303474, 0.38303474, 0.38303474, 0.38303474, 0.38303474,
        0.38303474, 0.11916945, 0.38303474, 0.11916945, 0.38303474,
        0.38303474], dtype=float64),
 Array([0.06057664, 0.04250809, 0.04

In [53]:
result.optimized_trainable_parameters["initial_params"]

[Array([0.15420315, 1.54598093], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]

In [54]:
result.returned_params

[[Array([[0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093],
         [0.15420315, 1.54598093]], dtype=float64),
  Array([[-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871],
         [-0.05085871]], dtype=float64),
  Array([[0.61438652],

In [55]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0000000233601563
fidelity of state 0: 0.20016072205698915
fidelity of state 1: 0.10516651778461271
fidelity of state 2: 0.17432247294473338
fidelity of state 3: 0.060243140237994326
fidelity of state 4: 0.060243140237994326
fidelity of state 5: 0.09456472955731363
fidelity of state 6: 0.20016072205698915
fidelity of state 7: 0.20016072205698915
fidelity of state 8: 0.20016072205698915
fidelity of state 9: 0.20016072205698915
fidelity of state 10: 0.20016072205698915
fidelity of state 11: 0.04513769456068037
fidelity of state 12: 0.20016072205698915
fidelity of state 13: 0.05850167527493088
fidelity of state 14: 0.20016072205698915
fidelity of state 15: 0.20016072205698915


### With Dissipation

In [56]:
# Note if tsave = jnp.linspace(0, 1, 1) = [0.0] then the decay is not applied ?
# because the first time step has the original non decayed state
# Question: Here I should not use any sort of Hamiltonian or tspan or something?
decay = Decay(
    c_ops=[tensor(identity(N_cav), jnp.sqrt(0.10) * sigmam())],
)
system_params = [decay, measure, qub_unitary, qub_cav]

for reward_weights in [[1.0,1.0],[0.0,1.0]]:
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-6,
        learning_rate=0.005,
        evo_type="density",
        batch_size=1,
        eval_batch_size=16,
        eval_time_steps=5,
    )

    print(f"reward weights: {reward_weights}\n fidelity@t=2: {result.fidelity_each_timestep[2]}\n fidelity_each_timestep: {jnp.mean(jnp.array(result.fidelity_each_timestep), axis=1)}\n")

reward weights: [1.0, 1.0]
 fidelity@t=2: [0.65061938 0.65061938 0.65061938 0.65061938 0.65061938 0.65061938
 0.65061938 0.65061938 0.65061938 0.65061938 0.65061938 0.65061938
 0.65061938 0.65061938 0.65061938 0.65061938]
 fidelity_each_timestep: [1.00000002 0.90450772 0.65061938 0.19914582 0.07253578 0.03681871]

reward weights: [0.0, 1.0]
 fidelity@t=2: [0.72274331 0.72274331 0.72274331 0.72274331 0.72274331 0.72274331
 0.72274331 0.72274331 0.72274331 0.72274331 0.72274331 0.72274331
 0.72274331 0.72274331 0.72274331 0.72274331]
 fidelity_each_timestep: [1.00000002 0.00198416 0.72274331 0.01281649 0.28514666 0.01568257]



In [57]:
result.fidelity_each_timestep

[Array([1.00000002, 1.00000002, 1.00000002, 1.00000002, 1.00000002,
        1.00000002, 1.00000002, 1.00000002, 1.00000002, 1.00000002,
        1.00000002, 1.00000002, 1.00000002, 1.00000002, 1.00000002,
        1.00000002], dtype=float64),
 Array([0.00198416, 0.00198416, 0.00198416, 0.00198416, 0.00198416,
        0.00198416, 0.00198416, 0.00198416, 0.00198416, 0.00198416,
        0.00198416, 0.00198416, 0.00198416, 0.00198416, 0.00198416,
        0.00198416], dtype=float64),
 Array([0.72274331, 0.72274331, 0.72274331, 0.72274331, 0.72274331,
        0.72274331, 0.72274331, 0.72274331, 0.72274331, 0.72274331,
        0.72274331, 0.72274331, 0.72274331, 0.72274331, 0.72274331,
        0.72274331], dtype=float64),
 Array([0.01545734, 0.01545734, 0.01545734, 0.01545734, 0.00841509,
        0.01545734, 0.00841509, 0.01545734, 0.01545734, 0.01545734,
        0.00841509, 0.01545734, 0.00841509, 0.00841509, 0.01545734,
        0.00841509], dtype=float64),
 Array([0.29841737, 0.29841737, 0.29

In [58]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0000000233601563
fidelity of state 0: 0.02436976844324975
fidelity of state 1: 0.02436976844324975
fidelity of state 2: 0.02436976844324975
fidelity of state 3: 0.02436976844324975
fidelity of state 4: 0.006217493294500122
fidelity of state 5: 0.02436976844324975
fidelity of state 6: 0.003632155144621505
fidelity of state 7: 0.018192130016481806
fidelity of state 8: 0.02436976844324975
fidelity of state 9: 0.00672605046208127
fidelity of state 10: 0.004079085139498669
fidelity of state 11: 0.02436976844324975
fidelity of state 12: 0.003632155144621505
fidelity of state 13: 0.01344404063516446
fidelity of state 14: 0.018192130016481806
fidelity of state 15: 0.006217493294500122


In [59]:
result.returned_params

[[Array([[3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00],
         [3.05732127e-09, 1.67551608e+00]], dtype=float64),
  Array([[0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.01158468],
         [0.0115846

In [60]:
result.optimized_trainable_parameters["initial_params"]

[Array([3.05732127e-09, 1.67551608e+00], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]